In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM 
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import pickle

In [ ]:
raw_data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1)raw_data

In [3]:
raw_data

,FIT101_0,LIT101_0,MV101_0,P101_0,P102_0,AIT201_0,AIT202_0,AIT203_0,FIT201_0,MV201_0,...,FIT504_3,P501_3,P502_3,PIT501_3,PIT502_3,PIT503_3,FIT601_3,P601_3,P602_3,P603_3
0,0.001538,0.331411,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000196,0.001268,0.000634,0.158997,0.001046,0.120169,8.121678e-08,0.000634,0.000634,0.000634
1,0.001551,0.331436,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000196,0.001267,0.000634,0.158940,0.001045,0.120055,8.118763e-08,0.000634,0.000634,0.000634
2,0.001578,0.331392,0.001268,0.001268,0.000634,0.166072,0.005321,0.208295,0.001548,0.001268,...,0.000195,0.001267,0.000633,0.158818,0.001045,0.120025,8.116716e-08,0.000633,0.000633,0.000633
3,0.001606,0.331431,0.001268,0.001268,0.000634,0.166054,0.005320,0.208273,0.001548,0.001268,...,0.000195,0.001267,0.000633,0.158715,0.001045,0.120023,8.116590e-08,0.000633,0.000633,0.000633
4,0.001628,0.331635,0.001267,0.001267,0.000634,0.165994,0.005318,0.208198,0.001548,0.001267,...,0.000195,0.001266,0.000633,0.158673,0.001045,0.119991,8.114453e-08,0.000633,0.000633,0.000633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449907,0.001691,0.327946,0.001270,0.001270,0.000635,0.106759,0.005489,0.191702,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159641,0.000600,0.120102,0.000000e+00,0.000635,0.000635,0.000635
449908,0.001690,0.328063,0.001270,0.001270,0.000635,0.106756,0.005489,0.191697,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159628,0.000600,0.120093,0.000000e+00,0.000635,0.000635,0.000635
449909,0.001673,0.328205,0.001270,0.001270,0.000635,0.106754,0.005489,0.191693,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159586,0.000600,0.120061,0.000000e+00,0.000635,0.000635,0.000635
449910,0.001668,0.328303,0.001270,0.001270,0.000635,0.106753,0.005489,0.191692,0.001563,0.001270,...,0.000195,0.001269,0.000635,0.159556,0.000549,0.120038,0.000000e+00,0.000635,0.000635,0.000635


In [ ]:
with open('labels_for_windowed_swat.pickle', 'rb') as f:
    y = pickle.load(f)[:449912]

In [ ]:
x_train = raw_data[y == 0.0][:200_000]

In [ ]:
train_test_split = x_train.index[-1]

In [17]:
train_test_split

211819

In [ ]:
x_test = raw_data[train_test_split:]

In [ ]:
y_test = y[train_test_split:]

In [ ]:
y_test.sum()

In [ ]:
def prepare_predict(predict):
    for i in range(len(predict)):
        if predict[i] == 1:
            predict[i] = 0
        else:
            predict[i] = 1
    return predict

In [ ]:
svm = OneClassSVM(nu=0.00458, kernel="rbf", gamma=0.0008, verbose=True)

In [12]:
%%time
svm.fit(x_train)

[LibSVM]CPU times: user 58.1 s, sys: 15.4 ms, total: 58.1 s
Wall time: 58.1 s


OneClassSVM(gamma=0.0008, nu=0.00458, verbose=True)

In [23]:
%%time
svm_predict = prepare_predict(svm.predict(x_test))

CPU times: user 33.8 s, sys: 19.9 ms, total: 33.8 s
Wall time: 33.8 s


In [59]:
print(classification_report(y_test, svm_predict))

              precision    recall  f1-score   support

           0       0.96      0.89      0.92    195187
           1       0.62      0.85      0.72     42906

    accuracy                           0.88    238093
   macro avg       0.79      0.87      0.82    238093
weighted avg       0.90      0.88      0.89    238093



In [ ]:
with open('svm.pickle', 'wb') as f:
    pickle.dump(svm, f)